## Installing if needed

In [ ]:

%pip install keras
%pip install sklearn
%pip install scikit-learn
%pip install tensorflow
%pip install matplotlib


## Imports

In [ ]:

import csv
import random
import tensorflow as tf

import numpy as np
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt



## Generating input 
This one creates data.csv with the combination of the results

In [ ]:

with open('merged_input_data.csv', 'w') as outputfile:
    #Select one csv to use as input, comment out the rest:
    
    #with open('board_all_noheader.csv', 'r') as resultsfile:
    with open('board_7_8_noheader.csv', 'r') as resultsfile:
    #with open('board_6_6_noheader.csv', 'r') as resultsfile:
        resultsreader = csv.reader(resultsfile)
        for row in resultsreader:
            filename_to_process = "rawdata\\"+row[0]
            trialcount = row[2] #OneFirstStrategy
            #trialcount = row[1] #BiggestFirstStrategy
            if( int(trialcount)>200000):
                trialcount="200000"


            result = ""
            with open(filename_to_process, "r") as individual_matrix_file:
                firstline = True
                for line in individual_matrix_file:
                    line = line.strip()
                    if line == "--------":
                        break
                    line_with_commas = ",".join(line)
                    if(firstline==True):
                        firstline=False
                    else:
                        line_with_commas = "\n" + line_with_commas


                    result += line_with_commas
            result = '"' + result + '",'+trialcount+"\n"
            outputfile.write(result)


## Creating the datasets
Input and output datasets

In [ ]:

#Az adatok be (x) és kimenetének (y) implementálása
x_data = []
y_data = []

with open('merged_input_data.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        array_string = row[0]
        number = int(row[1])
        matrix = np.array([[int(cell) for cell in line.split(',')] for line in array_string.split('\n')])
        rows, cols = matrix.shape
        #8x8-as mátrix létrehozása
        #TODO augmentalashoz mashova helyezni a paddingokat
        if rows < 8 or cols < 8:
            matrix = np.pad(matrix, ((0, max(0, 8 - rows)), (0, max(0, 8 - cols))), constant_values=1)

        x_data.append(matrix)
        y_data.append(number)
        #Az elforgatottjai is a mátrixnak a beolvasott adatok közé kerüljön
        for i in range(1, 4):
            rotated_matrix = np.rot90(matrix, i)
            x_data.append(rotated_matrix)
            y_data.append(number)


print("Full data size:")
print(len(x_train))


Splitting training, validation and test sets

In [ ]:


x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1) #tanuló és teszthalmaz lebontás 80-20%
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1) #tanuló és validációshalmaz lebontás 80-20%


## Building up model
Experiment with different layers and activation functions

In [ ]:

# A neurális háló architektúrájának létrehozása
model = tf.keras.Sequential()

#model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(8,8,1)))
model.add(Dense(512, input_shape=(8,8,1), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(8, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
#use dense 3 with softmax for categorization
#model.add(Dense(3, activation='softmax'))
#or instead use a linear activation for step count prediction
model.add(Dense(1))##

model.summary()


# A neurális háló veszteségfüggvénye. Select one case and comment out the rest in the next code block
#step count prediction case
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])


## Other codes supporting the classification

In [ ]:
#classification case
#model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Osztályok létrehozása
# Only for the classification problem
y_train_class = np.zeros((len(y_train), 3))
y_val_class = np.zeros((len(y_val), 3))
y_test_class = np.zeros((len(y_test), 3))

#Ha 100-nál kevesebb lépésből kirakható akkor könnyű, 200-nál többlépésnél nehéz és közte pedig közepes nehézségű a pálya
for i in range(len(y_train)):
    if y_train[i] < 500:
        y_train_class[i][0] = 1
    elif y_train[i] < 5000:
        y_train_class[i][2] = 2
    else:
        y_train_class[i][1] = 3

for i in range(len(y_val)):
    if y_val[i] < 500:
        y_val_class[i][0] = 1
    elif y_val[i] < 5000:
        y_val_class[i][2] = 2
    else:
        y_val_class[i][1] = 3

for i in range(len(y_test)):
    if y_test[i] < 500:
        y_test_class[i][0] = 1
    elif y_test[i] < 5000:
        y_test_class[i][2] = 2
    else:
        y_test_class[i][1] = 3


## Training the neural net

In [ ]:

#Adatok átformázása numpy tömbökké, valamint megfelelő formátum elérése
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
x_test = np.array(x_test)
y_test = np.array(y_test)

#x_train = x_train.reshape((x_train.shape[0], matrix.size))
#x_val = x_val.reshape((x_val.shape[0], matrix.size))
#x_test = x_test.reshape((x_test.shape[0], matrix.size))

# A neurális háló tanítása
early_stopping = EarlyStopping(monitor='val_loss', patience=20)



#in case of stepcount prediction:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=32, callbacks=[early_stopping])
#loss = model.evaluate(x_test, y_test)


#these are if we do classification, instead of step count prediction
#model.fit(x_train, y_train_class, validation_data=(x_val, y_val_class), epochs=500, callbacks=[early_stopping])
#loss = model.evaluate(x_test, y_test_class)


In [ ]:
# Véletlenszerűen kiválasztunk 10 indexet a tanítóhalmazból
random_indices = random.sample(range(len(x_train)), 10)
x_random = [x_train[i] for i in random_indices]
y_random = [y_train[i] for i in random_indices]

y_random_pred = model.predict(np.array(x_random))

for i in range(10):
    print(f"ITEM {i}: Expected {y_train[i]}, \n             got {y_random_pred[i][0]}")